In [4]:
import time 
import re
import pandas as pd 
from selenium import webdriver 
from selenium.webdriver import Chrome 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from webdriver_manager.chrome import ChromeDriverManager

# start by defining the options 
options = webdriver.ChromeOptions() 
options.headless = True # it's more scalable to work in headless mode 
# normally, selenium waits for all resources to download 
# we don't need it as the page also populated with the running javascript code. 
options.page_load_strategy = 'none' 
# this returns the path web driver downloaded 
chrome_path = ChromeDriverManager().install() 
chrome_service = Service(chrome_path) 
# pass the defined options and service objects to initialize the web driver 
driver = Chrome(options=options, service=chrome_service) 
driver.implicitly_wait(5)

# import beautiful soup
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer

In [5]:
page = 'https://diga.bfarm.de/de/verzeichnis'
driver.get(page) 
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
links = soup.find_all('a', class_='ember-view entity-app__button')
for i,j in enumerate(links):
    full_link = 'https://diga.bfarm.de' + j.get('href')
    links[i] = full_link
print(links)

['https://diga.bfarm.de/de/verzeichnis/00961', 'https://diga.bfarm.de/de/verzeichnis/01346', 'https://diga.bfarm.de/de/verzeichnis/00998', 'https://diga.bfarm.de/de/verzeichnis/00450', 'https://diga.bfarm.de/de/verzeichnis/01815', 'https://diga.bfarm.de/de/verzeichnis/00419', 'https://diga.bfarm.de/de/verzeichnis/01254', 'https://diga.bfarm.de/de/verzeichnis/01734', 'https://diga.bfarm.de/de/verzeichnis/01376', 'https://diga.bfarm.de/de/verzeichnis/01513', 'https://diga.bfarm.de/de/verzeichnis/01304', 'https://diga.bfarm.de/de/verzeichnis/01772', 'https://diga.bfarm.de/de/verzeichnis/00965', 'https://diga.bfarm.de/de/verzeichnis/01497', 'https://diga.bfarm.de/de/verzeichnis/00300', 'https://diga.bfarm.de/de/verzeichnis/01329', 'https://diga.bfarm.de/de/verzeichnis/00350', 'https://diga.bfarm.de/de/verzeichnis/01282', 'https://diga.bfarm.de/de/verzeichnis/00993', 'https://diga.bfarm.de/de/verzeichnis/01496', 'https://diga.bfarm.de/de/verzeichnis/00329', 'https://diga.bfarm.de/de/verzeic

In [6]:
# create dataframe with following columns
columns = 'app_name, developer_info, website_info, app_status, start_date, end_date, app_description, indication_codes, indication_names, price, property_values'
columns = columns.split(', ')
df = pd.DataFrame(columns=columns)

In [11]:
for link in links:
    # connect to a link
    driver.get(link)
    time.sleep(2)

    # find the button with more indications listed
    more_info_divs = driver.find_elements(By.CLASS_NAME, 'entity-app__info__more')
    more_info_div = more_info_divs[1]
    # click on the button called 'mehr erfahren'
    button = more_info_div.find_element(By.TAG_NAME, 'button')
    button.click()

    # get the html code
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    ## find app header information
    # Find the div element with the class "entity-app__header"
    header_div = soup.find('div', class_='entity-app__header')
    # Extract the app name
    app_name = header_div.find('h1', class_='entity-app__header__name').text.strip()
    # Extract the app developer and website information
    subheader = header_div.find('div', class_='entity-app__subheader')
    developer_info = subheader.find_all('span')[0].text.strip()
    website_info = subheader.find_all('a')[0].text.strip()

    ## find app status information
    # Find the div element with the class "entity-app__app-type"
    app_type_div = soup.find('div', class_='entity-app__app-type')
    # Extract the app status
    app_status = app_type_div.find('span').text.strip()
    # if app_status contains text 'vorläufig', then check for the date
    if 'Vorläufig' in app_status:
        # set status to 'vorläufig'
        app_status = 'vorläufig aufgenommen'
        # Extract the duration of the app type
        duration_text = app_type_div.find('small').text.strip()
        # remove ( from text
        start_date = duration_text.split('bis')[0].strip().replace('(', '')
        end_date = duration_text.split('bis')[1].strip().replace(')', '')
    else:
        start_date = None
        end_date = None


    ## find app description
    app_description = soup.find('div', class_='entity-app__summary').text.strip()

    ## find platform information
    # Find all of the div elements with the class "entity-app__info__list" within the info div
    list_divs = soup.find_all('div', class_='entity-app__info__list')

    platform_div = None
    application_div = None
    properties_div = None

    # Find the div element for each type of information
    for div in list_divs:
        header = div.find('div', class_='entity-app__info__list__header')
        if header:
            header_text = header.text.strip()
            if header_text == 'Plattformen':
                platform_div = div
            elif header_text == 'Anzuwenden bei':
                application_div = div
            elif header_text == 'Eigenschaften':
                properties_div = div

    # Extract the platform information
    platform_items = platform_div.find('div', class_='entity-app__info__list__items')
    platform_values = platform_items.find_all('div', class_='entity-app__info__list__items__value')
    platform_values = [i.text.strip() for i in platform_values]
    platform_values

    # Extract the application / indication / usage area information
    applications = soup.find('div', class_='modal-information__content__inner')
    applications = applications.find('div', class_='entity-app-answers__questions__answer')
    applications = applications.find('p')
    applications = str(applications).split('<br/>')
    applications
    indication_codes = []
    indication_names = []
    for i in applications:
        i = str(i).replace('<p>', '').replace('</p>', '')
        indication_codes.append(i.split(' ')[0])
        indication_names.append(i.split(' ', 1)[1])

    
    # Extract the properties information
    properties = {}
    properties_items = properties_div.find('div', class_='entity-app__info__list__items')
    property_values = properties_items.find_all('div', class_='entity-app__info__list__items__value')
    property_values = [i.text.strip() for i in property_values]
    # extract the numbers
    price = re.findall(r'\d{2,},\d{2}', property_values[0])
    if len(price) == 2:
        price1 = price[0].replace(',', '.')
        price2 = price[1].replace(',', '.')
        price = price1 + ' - ' + price2
    else:
        price = price[0].replace(',', '.')

    property_values = property_values[1:]
    df.loc[len(df)] = [app_name, developer_info, website_info, app_status, start_date, end_date, app_description, indication_codes, indication_names, price, property_values]

df

['399,84']
['718,20']
['345,10']
['210,00']
['178,50', '357,00']
178.50
<class 'str'>
['243,00']
['535,49']
['598,95']
['222,99']
['599,00']
['599,00']
['599,00']
['235,00']
['599,00']
['620,00']
['415,00']
['189,00']
['656,88']
['119,00']
['449,00']
['576,00']
['487,90']
['119,00', '329,00']
119.00
<class 'str'>
['249,00']
['952,00']
['426,96']
['535,50']
['784,21']
['217,18']
['479,52']
['690,00']
['224,99']
['230,00']
['499,80']
['239,96']
['192,01']
['499,80']


,app_name,developer_info,website_info,app_status,start_date,end_date,app_description,indication_codes,indication_names,price,property_values
0,CANKADO PRO-React Onco,"CANKADO GmbH, Deutschland",partners.cankado.com/about/,vorläufig aufgenommen,03.05.2021,02.05.2023,PRO-React Onco ist eine Web- und App-basierte ...,[C50],[Bösartige Neubildung der Brustdrüse [Mamma]],399.84,"[Keine Zusatzgeräte, Vertragsärztliche Leistun..."
1,Cara Care für Reizdarm,"HiDoc Technologies GmbH, Deutschland",cara.care/de/about/,vorläufig aufgenommen,26.12.2021,25.11.2023,Cara Care für Reizdarm ist eine digitale Gesun...,"[K58, K58.1, K58.2, K58.3, K58.8]","[Reizdarmsyndrom, Reizdarmsyndrom, Diarrhoe-pr...",718.20,"[Keine Zusatzgeräte, Keine vertragsärztlichen ..."
2,companion patella powered by medi - proved by ...,"PrehApp GmbH, Deutschland",prehapp.de,vorläufig aufgenommen,04.10.2021,03.03.2023,companion patella powered by medi - proved by ...,"[M22.2, M22.4, M76.5, M79.66, S83.0]","[Krankheiten im Patellofemoralbereich, Chondro...",345.10,"[Keine Zusatzgeräte, Vertragsärztliche Leistun..."
3,deprexis,"GAIA AG, Deutschland",gaia-group.com/de/,Dauerhaft aufgenommen,None,None,deprexis ist ein interaktives onlinebasiertes ...,"[F32.0, F32.1, F32.2, F33.0, F33.1, F33.2]","[Leichte depressive Episode, Mittelgradige dep...",210.00,"[Keine Zusatzgeräte, Keine vertragsärztlichen ..."
4,edupression.com®,"SOFY GmbH, Österreich",edupression.com/,vorläufig aufgenommen,26.12.2022,25.08.2023,Die DiGA edupression.com® besteht aus dem Medi...,"[F32.0, F32.1, F33.0, F33.1]","[Leichte depressive Episode, Mittelgradige dep...",178.50 - 357.00,"[Keine Zusatzgeräte, Keine vertragsärztlichen ..."
5,elevida,"GAIA AG, Deutschland",gaia-group.com/de/,Dauerhaft aufgenommen,None,None,elevida ist eine digitale Gesundheitsanwendung...,[G35],[Multiple Sklerose [Encephalomyelitis dissemin...,243.00,"[Keine Zusatzgeräte, Keine vertragsärztlichen ..."
6,elona therapy Depression,"Elona Health GmbH, Deutschland",www.elona.health,vorläufig aufgenommen,26.12.2022,25.12.2023,elona therapy Depression ist die digitale Gesu...,"[F32.0, F32.1, F32.2, F33.0, F33.1, F33.2, F34.1]","[Leichte depressive Episode, Mittelgradige dep...",535.49,"[Keine Zusatzgeräte, Vertragsärztliche Leistun..."
7,Endo-App,"Endo Health GmbH, Deutschland",diga.endometriose.app,vorläufig aufgenommen,09.10.2022,08.10.2023,Die Endo-App ist ein digitales Medizinprodukt ...,[N80],[Endometriose],598.95,"[Keine Zusatzgeräte, Keine vertragsärztlichen ..."
8,HelloBetter Diabetes und Depression,GET.ON Institut für Online Gesundheitstraining...,hellobetter.de/,Dauerhaft aufgenommen,None,None,HelloBetter Diabetes und Depression ist ein in...,"[E10, E11]","[Diabetes mellitus, Typ 1, Diabetes mellitus, ...",222.99,"[Keine Zusatzgeräte, Keine vertragsärztlichen ..."
9,HelloBetter Panik,GET.ON Institut für Online Gesundheitstraining...,hellobetter.de/,Dauerhaft aufgenommen,None,None,HelloBetter Panik ist ein interaktives psychol...,"[F40.01, F41.0]","[Agoraphobie: Mit Panikstörung, Panikstörung [...",599.00,"[Keine Zusatzgeräte, Keine vertragsärztlichen ..."


In [12]:
# remove [ ] from indication codes
df['indication_codes'] = df['indication_codes'].apply(lambda x: str(x).replace('[', '').replace(']', ''))
# remove ' from indication codes
df['indication_codes'] = df['indication_codes'].apply(lambda x: str(x).replace("'", ''))
# remove [ ] from property values
df['property_values'] = df['property_values'].apply(lambda x: str(x).replace('[', '').replace(']', '').replace("'", ''))
# remove [ ] from indication names
df['indication_names'] = df['indication_names'].apply(lambda x: str(x).replace('[', '').replace(']', '').replace("'", ''))


In [15]:
# save df to csv
df.to_csv('diga.csv', index=False)

In [6]:
# load df from csv
import pandas as pd
df = pd.read_csv('diga.csv')
df

,app_name,developer_info,website_info,app_status,start_date,end_date,app_description,indication_codes,indication_names,price,property_values
0,CANKADO PRO-React Onco,"CANKADO GmbH, Deutschland",partners.cankado.com/about/,vorläufig aufgenommen,03.05.2021,02.05.2023,PRO-React Onco ist eine Web- und App-basierte ...,C50,Bösartige Neubildung der Brustdrüse Mamma,399.84,"Keine Zusatzgeräte, Vertragsärztliche Leistung..."
1,Cara Care für Reizdarm,"HiDoc Technologies GmbH, Deutschland",cara.care/de/about/,vorläufig aufgenommen,26.12.2021,25.11.2023,Cara Care für Reizdarm ist eine digitale Gesun...,"K58, K58.1, K58.2, K58.3, K58.8","Reizdarmsyndrom, Reizdarmsyndrom, Diarrhoe-prä...",718.20,"Keine Zusatzgeräte, Keine vertragsärztlichen L..."
2,companion patella powered by medi - proved by ...,"PrehApp GmbH, Deutschland",prehapp.de,vorläufig aufgenommen,04.10.2021,03.03.2023,companion patella powered by medi - proved by ...,"M22.2, M22.4, M76.5, M79.66, S83.0","Krankheiten im Patellofemoralbereich, Chondrom...",345.10,"Keine Zusatzgeräte, Vertragsärztliche Leistung..."
3,deprexis,"GAIA AG, Deutschland",gaia-group.com/de/,Dauerhaft aufgenommen,NaN,NaN,deprexis ist ein interaktives onlinebasiertes ...,"F32.0, F32.1, F32.2, F33.0, F33.1, F33.2","Leichte depressive Episode, Mittelgradige depr...",210.00,"Keine Zusatzgeräte, Keine vertragsärztlichen L..."
4,edupression.com®,"SOFY GmbH, Österreich",edupression.com/,vorläufig aufgenommen,26.12.2022,25.08.2023,Die DiGA edupression.com® besteht aus dem Medi...,"F32.0, F32.1, F33.0, F33.1","Leichte depressive Episode, Mittelgradige depr...",178.50 - 357.00,"Keine Zusatzgeräte, Keine vertragsärztlichen L..."
5,elevida,"GAIA AG, Deutschland",gaia-group.com/de/,Dauerhaft aufgenommen,NaN,NaN,elevida ist eine digitale Gesundheitsanwendung...,G35,Multiple Sklerose Encephalomyelitis disseminata,243.00,"Keine Zusatzgeräte, Keine vertragsärztlichen L..."
6,elona therapy Depression,"Elona Health GmbH, Deutschland",www.elona.health,vorläufig aufgenommen,26.12.2022,25.12.2023,elona therapy Depression ist die digitale Gesu...,"F32.0, F32.1, F32.2, F33.0, F33.1, F33.2, F34.1","Leichte depressive Episode, Mittelgradige depr...",535.49,"Keine Zusatzgeräte, Vertragsärztliche Leistung..."
7,Endo-App,"Endo Health GmbH, Deutschland",diga.endometriose.app,vorläufig aufgenommen,09.10.2022,08.10.2023,Die Endo-App ist ein digitales Medizinprodukt ...,N80,Endometriose,598.95,"Keine Zusatzgeräte, Keine vertragsärztlichen L..."
8,HelloBetter Diabetes und Depression,GET.ON Institut für Online Gesundheitstraining...,hellobetter.de/,Dauerhaft aufgenommen,NaN,NaN,HelloBetter Diabetes und Depression ist ein in...,"E10, E11","Diabetes mellitus, Typ 1, Diabetes mellitus, T...",222.99,"Keine Zusatzgeräte, Keine vertragsärztlichen L..."
9,HelloBetter Panik,GET.ON Institut für Online Gesundheitstraining...,hellobetter.de/,Dauerhaft aufgenommen,NaN,NaN,HelloBetter Panik ist ein interaktives psychol...,"F40.01, F41.0","Agoraphobie: Mit Panikstörung, Panikstörung ep...",599.00,"Keine Zusatzgeräte, Keine vertragsärztlichen L..."


In [19]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

query_embedding = model.encode('How big is London')
passage_embedding1 = model.encode(['London has 9,787,426 inhabitants at the 2011 census',
                                  'London is known for its finacial district'])

passage_embedding2 = model.encode(['The greatest pleasure of a man is his worth and his honor, they way his compatriots look at thim.'])
passage_embedding3 = model.encode(['London is of similiar size to Berlin'])

print("Similarity:", util.dot_score(query_embedding, passage_embedding1))

Similarity: tensor([[0.5472, 0.6330]])


In [20]:
query = model.encode('rückenschmerzen')
passage = model.encode(['HelloBetter ratiopharm chronischer Schmerz ist ein interaktives psychologisches Online-Programm zur anhaltenden Reduktion der Schmerzbeeinträchtigung bei Menschen mit chronischen Schmerzen. Neben fundierter Psychoedukation mittels Texten, Videos und Audios, erlernen Betroffene wirksame Strategien der Akzeptanz- und Commitment-Therapie (ACT), die eine Weiterentwicklung der kognitiven Verhaltenstherapie (KVT) darstellt. Diese Strategien können durch praktische Übungen in den Alltag übertragen werden.', 'HelloBetter Schlafen ist ein interaktives psychologisches Therapieprogramm zur Reduktion insomnischer Beschwerden. Das Online-Programm basiert auf der kognitiven Verhaltenstherapie für Insomnie (KVT-I) und wurde von Expertinnen und Experten aus der Wissenschaft, Psychologie und Psychotherapie sowie gemeinsam mit Betroffenen entwickelt. Neben fundierter Psychoedukation vermittelt das Online-Programm wirksame Strategien aus der kognitiven Verhaltenstherapie nach dem aktuellsten Stand wissenschaftlicher Erkenntnisse. Dazu gehören unter anderem Schlafhygiene, Bettzeitverkürzung, Stimuluskontrolle, kognitive Strategien, Entspannungsverfahren, eine Rückfallprophylaxe sowie ein fortlaufendes Online-Tagebuch.'])
print("Similarity:", util.dot_score(query, passage))

Similarity: tensor([[0.2911, 0.2449]])


In [18]:
model = SentenceTransformer('Sahajtomar/German-semantic')
query = model.encode('rückenschmerzen')
passage = model.encode(['HelloBetter ratiopharm chronischer Schmerz ist ein interaktives psychologisches Online-Programm zur anhaltenden Reduktion der Schmerzbeeinträchtigung bei Menschen mit chronischen Schmerzen. Neben fundierter Psychoedukation mittels Texten, Videos und Audios, erlernen Betroffene wirksame Strategien der Akzeptanz- und Commitment-Therapie (ACT), die eine Weiterentwicklung der kognitiven Verhaltenstherapie (KVT) darstellt. Diese Strategien können durch praktische Übungen in den Alltag übertragen werden.', 'HelloBetter Schlafen ist ein interaktives psychologisches Therapieprogramm zur Reduktion insomnischer Beschwerden. Das Online-Programm basiert auf der kognitiven Verhaltenstherapie für Insomnie (KVT-I) und wurde von Expertinnen und Experten aus der Wissenschaft, Psychologie und Psychotherapie sowie gemeinsam mit Betroffenen entwickelt. Neben fundierter Psychoedukation vermittelt das Online-Programm wirksame Strategien aus der kognitiven Verhaltenstherapie nach dem aktuellsten Stand wissenschaftlicher Erkenntnisse. Dazu gehören unter anderem Schlafhygiene, Bettzeitverkürzung, Stimuluskontrolle, kognitive Strategien, Entspannungsverfahren, eine Rückfallprophylaxe sowie ein fortlaufendes Online-Tagebuch.'])
print("Similarity:", util.dot_score(query, passage))

Similarity: tensor([[108.3415,  94.2840]])


In [ ]:
from transformers import AutoTokenizer, AutoModel

# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode the text data
encoded_text = tokenizer.batch_encode_plus(df['text'], return_tensors='pt')
encoded_text = model(**encoded_text)[0]

# Find the most similar entries in the dataframe using cosine similarity
query_vector = encoded_text[0]  # select the first entry as the query
similarity = encoded_text @ query_vector.T  # calculate the cosine similarity
most_similar_index = similarity.argmax().item()  # find the index of the most similar entry
most_similar_entry = df.iloc[most_similar_index]  # retrieve the most similar entry from the dataframe

print(f'Most similar entry: {most_similar_entry["text"]}')
